In [3]:
import xarray as xr
import pandas as pd
import os

## Prepare data

In [6]:

import numpy as np

home_dir = "/Users/bean/Documents/Doctorate/1Research/Polaris/sedona_experiments"
ds = xr.open_dataset(os.path.join(home_dir, "yearly_data/t2m_3mo_data.nc"))
ds = ds.rename({"valid_time": "time"})
ds

<xarray.Dataset> Size: 9GB
Dimensions:    (time: 2160, latitude: 721, longitude: 1440)
Coordinates:
    number     int64 8B ...
  * time       (time) datetime64[ns] 17kB 2023-01-01 ... 2023-03-31T23:00:00
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    expver     (time) <U4 35kB ...
Data variables:
    t2m        (time, latitude, longitude) float32 9GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-24T18:19 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
# Loop over every 24th index in the time dimension
for i in range(0, ds.sizes['time'], 24):
    ds_slice = ds.isel(time=i)
    date = pd.to_datetime(ds_slice.time.values).strftime('%Y%m%dT%H%M')[:10]
    ds_slice.expand_dims("time").to_netcdf(os.path.join(home_dir, f"daily_data/t2m-{date}.nc"))

In [40]:
from pathlib import Path

folder_path = Path(os.path.join(home_dir, "daily_data"))
nc_files = list(folder_path.glob("*.nc"))
nc_files

[PosixPath('/Users/bean/Documents/Doctorate/1Research/Polaris/sedona_experiments/daily_data/t2m-2023-01-04.nc'),
 PosixPath('/Users/bean/Documents/Doctorate/1Research/Polaris/sedona_experiments/daily_data/t2m-2023-01-10.nc'),
 PosixPath('/Users/bean/Documents/Doctorate/1Research/Polaris/sedona_experiments/daily_data/t2m-2023-01-01.nc'),
 PosixPath('/Users/bean/Documents/Doctorate/1Research/Polaris/sedona_experiments/daily_data/t2m-2023-01-05.nc'),
 PosixPath('/Users/bean/Documents/Doctorate/1Research/Polaris/sedona_experiments/daily_data/t2m-20230101T0.nc'),
 PosixPath('/Users/bean/Documents/Doctorate/1Research/Polaris/sedona_experiments/daily_data/t2m-2023-01-08.nc'),
 PosixPath('/Users/bean/Documents/Doctorate/1Research/Polaris/sedona_experiments/daily_data/t2m-2023-01-09.nc'),
 PosixPath('/Users/bean/Documents/Doctorate/1Research/Polaris/sedona_experiments/daily_data/t2m-2023-01-02.nc'),
 PosixPath('/Users/bean/Documents/Doctorate/1Research/Polaris/sedona_experiments/daily_data/t2m-

In [41]:
file = nc_files[0]
ds = xr.open_dataset(file)
ds

<xarray.Dataset> Size: 4MB
Dimensions:    (latitude: 721, longitude: 1440)
Coordinates:
    number     int64 8B ...
    time       datetime64[ns] 8B ...
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    expver     <U4 16B ...
Data variables:
    t2m        (latitude, longitude) float32 4MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-24T18:19 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:

# for f in file:
ds = xr.open_dataset(os.path.join(home_dir, "daily_data/t2m-2023-01-01.nc"))
df = ds['t2m'].to_dataframe().reset_index()
df.rename(columns={'t2m': '2m_temperature'}, inplace=True)
df

file_path = os.path.join(home_dir, "formatted_data")
try:
    df.to_parquet(os.path.join(file_path, "test.parquet"))
except Exception as e:
    print(f"Parquet failed: {e}, falling back to CSV.")
    df.to_csv(os.path.join(file_path, "test.csv"), index=False)

Yuchuan's code: `iharp3/sedona-exp/0508.ipynb`

In [24]:
# Y.H. version
from sedona.spark import SedonaContext

config = SedonaContext.builder(). \
    config('spark.jars.repositories', 'https://artifacts.unidata.ucar.edu/repository/unidata-all/'). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-3.5_2.12:1.7.1,'         # 3.3_2.12:1.7.1 in documentation
           'org.datasyslab:geotools-wrapper:1.7.1-28.5,'
           'edu.ucar:cdm-core:5.4.2'). \
    config('spark.driver.memory', '10g'). \
    config('spark.driver.maxResultSize', '5g'). \
    config('spark.network.timeout', '1000s'). \
    getOrCreate()

sedona = SedonaContext.create(config)

25/05/24 19:42:11 WARN UDTRegistration: Cannot register UDT for org.geotools.coverage.grid.GridCoverage2D, which is already registered.
25/05/24 19:42:11 WARN SimpleFunctionRegistry: The function rs_union_aggr replaced a previously registered function.
25/05/24 19:42:11 WARN UDTRegistration: Cannot register UDT for org.locationtech.jts.geom.Geometry, which is already registered.
25/05/24 19:42:11 WARN UDTRegistration: Cannot register UDT for org.apache.sedona.common.geometryObjects.Geography, which is already registered.
25/05/24 19:42:11 WARN UDTRegistration: Cannot register UDT for org.locationtech.jts.index.SpatialIndex, which is already registered.
25/05/24 19:42:11 WARN SimpleFunctionRegistry: The function st_envelope_aggr replaced a previously registered function.
25/05/24 19:42:11 WARN SimpleFunctionRegistry: The function st_intersection_aggr replaced a previously registered function.
25/05/24 19:42:11 WARN SimpleFunctionRegistry: The function st_union_aggr replaced a previously

In [25]:
df = sedona.read.format("binaryFile").load(os.path.join(home_dir, "daily_data/t2m-2023-01-*.nc"))
df.show()

+--------------------+--------------------+-------+--------------------+
|                path|    modificationTime| length|             content|
+--------------------+--------------------+-------+--------------------+
|file:/Users/bean/...|2025-05-24 16:49:...|1534282|[89 48 44 46 0D 0...|
|file:/Users/bean/...|2025-05-24 16:49:...|1534186|[89 48 44 46 0D 0...|
|file:/Users/bean/...|2025-05-24 16:49:...|1534057|[89 48 44 46 0D 0...|
|file:/Users/bean/...|2025-05-24 16:49:...|1533211|[89 48 44 46 0D 0...|
|file:/Users/bean/...|2025-05-24 16:49:...|1532632|[89 48 44 46 0D 0...|
|file:/Users/bean/...|2025-05-24 16:48:...|1531257|[89 48 44 46 0D 0...|
|file:/Users/bean/...|2025-05-24 16:49:...|1530703|[89 48 44 46 0D 0...|
|file:/Users/bean/...|2025-05-24 16:49:...|1529510|[89 48 44 46 0D 0...|
|file:/Users/bean/...|2025-05-24 16:48:...|1527674|[89 48 44 46 0D 0...|
|file:/Users/bean/...|2025-05-24 16:48:...|1527537|[89 48 44 46 0D 0...|
+--------------------+--------------------+-------+

In [12]:
from pyspark.sql import functions as f
df = df.withColumn("raster", f.expr("RS_FromNetCDF(content, 't2m', 'longitude', 'latitude')"))
df.show()

+--------------------+--------------------+-------+--------------------+--------------------+
|                path|    modificationTime| length|             content|              raster|
+--------------------+--------------------+-------+--------------------+--------------------+
|file:/Users/bean/...|2025-05-24 16:49:...|1534282|[89 48 44 46 0D 0...|GridCoverage2D["g...|
|file:/Users/bean/...|2025-05-24 16:49:...|1534186|[89 48 44 46 0D 0...|GridCoverage2D["g...|
|file:/Users/bean/...|2025-05-24 16:49:...|1534057|[89 48 44 46 0D 0...|GridCoverage2D["g...|
|file:/Users/bean/...|2025-05-24 16:49:...|1533211|[89 48 44 46 0D 0...|GridCoverage2D["g...|
|file:/Users/bean/...|2025-05-24 16:49:...|1532632|[89 48 44 46 0D 0...|GridCoverage2D["g...|
|file:/Users/bean/...|2025-05-24 16:48:...|1531257|[89 48 44 46 0D 0...|GridCoverage2D["g...|
|file:/Users/bean/...|2025-05-24 16:49:...|1530703|[89 48 44 46 0D 0...|GridCoverage2D["g...|
|file:/Users/bean/...|2025-05-24 16:49:...|1529510|[89 48 44

In [13]:
df.createOrReplaceTempView("NetCDF")
sedona.sql("SELECT RS_NetCDFInfo(content) as band FROM NetCDF").show(truncate=False)

+---------------------------------+
|band                             |
+---------------------------------+
|t2m(latitude=721, longitude=1440)|
|t2m(latitude=721, longitude=1440)|
|t2m(latitude=721, longitude=1440)|
|t2m(latitude=721, longitude=1440)|
|t2m(latitude=721, longitude=1440)|
|t2m(latitude=721, longitude=1440)|
|t2m(latitude=721, longitude=1440)|
|t2m(latitude=721, longitude=1440)|
|t2m(latitude=721, longitude=1440)|
|t2m(latitude=721, longitude=1440)|
+---------------------------------+



In [14]:
sedona.sql("SELECT RS_MetaData(raster) FROM NetCDF").show(truncate=False)

+-------------------------------------------------------------------+
|rs_metadata(raster)                                                |
+-------------------------------------------------------------------+
|{-0.125, 90.125, 1440, 721, 0.25, -0.25, 0.0, 0.0, 0, 1, 1440, 721}|
|{-0.125, 90.125, 1440, 721, 0.25, -0.25, 0.0, 0.0, 0, 1, 1440, 721}|
|{-0.125, 90.125, 1440, 721, 0.25, -0.25, 0.0, 0.0, 0, 1, 1440, 721}|
|{-0.125, 90.125, 1440, 721, 0.25, -0.25, 0.0, 0.0, 0, 1, 1440, 721}|
|{-0.125, 90.125, 1440, 721, 0.25, -0.25, 0.0, 0.0, 0, 1, 1440, 721}|
|{-0.125, 90.125, 1440, 721, 0.25, -0.25, 0.0, 0.0, 0, 1, 1440, 721}|
|{-0.125, 90.125, 1440, 721, 0.25, -0.25, 0.0, 0.0, 0, 1, 1440, 721}|
|{-0.125, 90.125, 1440, 721, 0.25, -0.25, 0.0, 0.0, 0, 1, 1440, 721}|
|{-0.125, 90.125, 1440, 721, 0.25, -0.25, 0.0, 0.0, 0, 1, 1440, 721}|
|{-0.125, 90.125, 1440, 721, 0.25, -0.25, 0.0, 0.0, 0, 1, 1440, 721}|
+-------------------------------------------------------------------+



In [26]:
sedona.stop()

---

In [29]:
import xarray as xr
import pandas as pd
from sedona.spark import SedonaContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

# Open dataset using xarray
ds = xr.open_dataset(os.path.join(home_dir, "daily_data/t2m-2023-01-01.nc"))
df = ds['t2m'].to_dataframe().reset_index()
df.rename(columns={'t2m': '2m_temperature'}, inplace=True)

# Start Spark + Sedona
config = SedonaContext.builder(). \
    config('spark.jars.repositories', 'https://artifacts.unidata.ucar.edu/repository/unidata-all/'). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-3.5_2.12:1.7.1,'         # 3.3_2.12:1.7.1 in documentation
           'org.datasyslab:geotools-wrapper:1.7.1-28.5,'
           'edu.ucar:cdm-core:5.4.2'). \
    config('spark.driver.memory', '10g'). \
    config('spark.driver.maxResultSize', '5g'). \
    config('spark.network.timeout', '1000s'). \
    getOrCreate()
sedona = SedonaContext.create(config)

# Convert Pandas DataFrame to Spark DataFrame
sdf = config.createDataFrame(df)

# Create a geometry column for spatial queries
sdf = sdf.withColumn("geom", expr("ST_Point(cast(longitude as double), cast(latitude as double))"))


25/05/24 19:46:23 WARN UDTRegistration: Cannot register UDT for org.geotools.coverage.grid.GridCoverage2D, which is already registered.
25/05/24 19:46:23 WARN UDTRegistration: Cannot register UDT for org.locationtech.jts.geom.Geometry, which is already registered.
25/05/24 19:46:23 WARN UDTRegistration: Cannot register UDT for org.apache.sedona.common.geometryObjects.Geography, which is already registered.
25/05/24 19:46:23 WARN UDTRegistration: Cannot register UDT for org.locationtech.jts.index.SpatialIndex, which is already registered.


In [30]:
sdf.show()

25/05/24 19:47:26 WARN TaskSetManager: Stage 3 contains a task of very large size (6094 KiB). The maximum recommended task size is 1000 KiB.


+--------+---------+------+-------------------+------+------------------+---------------+
|latitude|longitude|number|               time|expver|    2m_temperature|           geom|
+--------+---------+------+-------------------+------+------------------+---------------+
|    90.0|      0.0|     0|2023-01-01 00:00:00|  0001|247.93197631835938|   POINT (0 90)|
|    90.0|     0.25|     0|2023-01-01 00:00:00|  0001|247.93197631835938|POINT (0.25 90)|
|    90.0|      0.5|     0|2023-01-01 00:00:00|  0001|247.93197631835938| POINT (0.5 90)|
|    90.0|     0.75|     0|2023-01-01 00:00:00|  0001|247.93197631835938|POINT (0.75 90)|
|    90.0|      1.0|     0|2023-01-01 00:00:00|  0001|247.93197631835938|   POINT (1 90)|
|    90.0|     1.25|     0|2023-01-01 00:00:00|  0001|247.93197631835938|POINT (1.25 90)|
|    90.0|      1.5|     0|2023-01-01 00:00:00|  0001|247.93197631835938| POINT (1.5 90)|
|    90.0|     1.75|     0|2023-01-01 00:00:00|  0001|247.93197631835938|POINT (1.75 90)|
|    90.0|

25/05/24 19:47:31 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 3 (TID 2): Attempting to kill Python Worker


In [31]:
sdf.createOrReplaceTempView("temperature_spatial")

In [45]:
##### Get variable query #####
min_lat = 80
max_lat = 90
min_lon = 0
max_lon = 2
start_time = "2022-01-01 00:00:00"
end_time = "2023-01-02- 00:00:00"


query = f"""
    SELECT time, latitude, longitude, `2m_temperature`
    FROM temperature_spatial
    WHERE latitude BETWEEN {min_lat} AND {max_lat}
        AND longitude BETWEEN {min_lon} AND {max_lon} 
    """


result = sedona.sql(query)
result.show()

25/05/24 21:06:28 WARN TaskSetManager: Stage 10 contains a task of very large size (6094 KiB). The maximum recommended task size is 1000 KiB.


+-------------------+--------+---------+------------------+
|               time|latitude|longitude|    2m_temperature|
+-------------------+--------+---------+------------------+
|2023-01-01 00:00:00|    90.0|      0.0|247.93197631835938|
|2023-01-01 00:00:00|    90.0|     0.25|247.93197631835938|
|2023-01-01 00:00:00|    90.0|      0.5|247.93197631835938|
|2023-01-01 00:00:00|    90.0|     0.75|247.93197631835938|
|2023-01-01 00:00:00|    90.0|      1.0|247.93197631835938|
|2023-01-01 00:00:00|    90.0|     1.25|247.93197631835938|
|2023-01-01 00:00:00|    90.0|      1.5|247.93197631835938|
|2023-01-01 00:00:00|    90.0|     1.75|247.93197631835938|
|2023-01-01 00:00:00|    90.0|      2.0|247.93197631835938|
|2023-01-01 00:00:00|   89.75|      0.0|248.90853881835938|
|2023-01-01 00:00:00|   89.75|     0.25|248.91049194335938|
|2023-01-01 00:00:00|   89.75|      0.5|248.91244506835938|
|2023-01-01 00:00:00|   89.75|     0.75|248.91439819335938|
|2023-01-01 00:00:00|   89.75|      1.0|

25/05/24 21:06:32 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 10 (TID 21): Attempting to kill Python Worker


In [ ]:
query = f"""
    SELECT get_date(time), latitude, longitude, max(`2m_temperature`)
    FROM temperature_spatial
    WHERE latitude BETWEEN {min_lat} AND {max_lat}
        AND longitude BETWEEN {min_lon} AND {max_lon} 
    GROUPBY get_date(time <- timestamp) , latitude , longitude

    """

result = sedona.sql(query)
result.show()

In [ ]:
sedona.stop()